In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("CurzonBikeMakes").getOrCreate())

In [3]:
from pyspark.sql.functions import col, expr

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, DoubleType, LongType, IntegerType

In [5]:
company_info_path = "D:\Spark Projects\Curzon Bike Makes\CurzonBikeMakes\SparkBasictryoutdatasets\CompanyInfo.csv"
company_orders_path = "D:\Spark Projects\Curzon Bike Makes\CurzonBikeMakes\SparkBasictryoutdatasets\CompanyOrders.csv"

In [6]:
schema_orders = StructType([
    StructField("OrderID", StringType(), True),
    StructField("CompanyID", StringType(), True),
    StructField("CompanyName", StringType(), True),
    StructField("CompanyLocation", StringType(), True),
    StructField("ComponentToBeManufactured", StringType(), True),
    StructField("Quantity", LongType(), True),
    StructField("EstimatedCost", DoubleType(), True),
    StructField("OrderDate", StringType(), True),
    StructField("DueDate", StringType(), True),
    StructField("CompletionStatus", StringType(), True),
    StructField("DeliveryStatus", StringType(), True)
])

schema_info = StructType([
    StructField("CompanyID", StringType(), True),
    StructField("CompanyName", StringType(), True),
    StructField("CompanyLocation", StringType(), True),
    StructField("CompanyAddress", StringType(), True),
    StructField("CompanyContact", LongType(), True),
    StructField("ProfitMargin", DoubleType(), True),
    StructField("EstablishedYear", IntegerType(), True)
])

In [7]:
company_info_df = (spark.read.format("csv").option("header", False).schema(schema=schema_info).load(company_info_path))
company_orders_df =(spark.read.format("csv").option("header", False).schema(schema=schema_orders).load(company_orders_path))

In [8]:
company_orders_df.show(10)

+-------+---------+----------------+---------------+-------------------------+--------+-------------+---------+---------+----------------+--------------+
|OrderID|CompanyID|     CompanyName|CompanyLocation|ComponentToBeManufactured|Quantity|EstimatedCost|OrderDate|  DueDate|CompletionStatus|DeliveryStatus|
+-------+---------+----------------+---------------+-------------------------+--------+-------------+---------+---------+----------------+--------------+
|OD10001|    C1108|     Raido Bikes|        Georgia|             Spring Forks|   21103|   9487867.48| 3-Nov-19|24-Dec-19|         Pending|       Pending|
|OD10002|    C1035| Ilmio Jane Moto| North Carolina|                   Fender|   26272|   4899369.32|18-Dec-19|14-Mar-20|         Pending|       Pending|
|OD10003|    C1174|   MotoLab Bikes|         Nevada|                Generator|   39789|   4008848.49| 7-Dec-19|31-Jan-20|       Completed|     Delivered|
|OD10004|    C1121|        Lee Auto|         Oregon|                   Brake

In [9]:
company_info_df.show(10)

+---------+----------------+---------------+-------------------+--------------+------------+---------------+
|CompanyID|     CompanyName|CompanyLocation|     CompanyAddress|CompanyContact|ProfitMargin|EstablishedYear|
+---------+----------------+---------------+-------------------+--------------+------------+---------------+
|    C1001|    Steel Wheels|   Pennsylvania|   80 Stang Parkway|    7172809057|  4061361.08|           2007|
|    C1002|   MotoLab Bikes|          Texas|    1 Anthes Avenue|    6829309477|  2733417.09|           2011|
|    C1003|        Lee Auto|     California|  87985 Linden Pass|    6193601290|  4422985.26|           1994|
|    C1004|     Zoom Dynamo|          Texas|56 La Follette Pass|    8327309957|  3102258.84|           1993|
|    C1005|Quakesons Motors|          Texas| 8 Briar Crest Pass|    8175902345|  3584342.27|           2002|
|    C1006|    Steel Wheels|   Pennsylvania|   035 Iowa Terrace|    5706889534|  3169758.24|           2009|
|    C1007|     Rai

##### Details Of all Companies that are atleast 20 years old

In [10]:
company_info_df.where(col("EstablishedYear") <= 2002).show()

+---------+------------------+--------------------+--------------------+--------------+------------+---------------+
|CompanyID|       CompanyName|     CompanyLocation|      CompanyAddress|CompanyContact|ProfitMargin|EstablishedYear|
+---------+------------------+--------------------+--------------------+--------------+------------+---------------+
|    C1003|          Lee Auto|          California|   87985 Linden Pass|    6193601290|  4422985.26|           1994|
|    C1004|       Zoom Dynamo|               Texas| 56 La Follette Pass|    8327309957|  3102258.84|           1993|
|    C1005|  Quakesons Motors|               Texas|  8 Briar Crest Pass|    8175902345|  3584342.27|           2002|
|    C1007|       Raido Bikes|              Hawaii|    3925 Clove Drive|    8086946394|  2856030.07|           2001|
|    C1011|      Steel Wheels|             Arizona|83 Merchant Junction|    6026446720|  2025983.74|           1992|
|    C1013|      Craiko Roads|                Iowa|   5 Bowman J

In [11]:
companies_20_yr_old = company_info_df.where(col("EstablishedYear") <= 2002).count()

In [12]:
companies_20_yr_old

92

##### Fetch the name of the company which has the highest profit margin among those who have ordered Shock Absorbers. 

In [13]:
company_orders_df.where(col("ComponentToBeManufactured") == 'Shock Absorber').show()

+-------+---------+------------------+--------------------+-------------------------+--------+-------------+---------+---------+----------------+--------------+
|OrderID|CompanyID|       CompanyName|     CompanyLocation|ComponentToBeManufactured|Quantity|EstimatedCost|OrderDate|  DueDate|CompletionStatus|DeliveryStatus|
+-------+---------+------------------+--------------------+-------------------------+--------+-------------+---------+---------+----------------+--------------+
|OD10056|    C1047|      Damson Bikes|             Florida|           Shock Absorber|   55552|   9014369.62|19-Dec-19|30-Mar-20|       Completed|     Delivered|
|OD10067|    C1115|      Craiko Roads|             Florida|           Shock Absorber|   22505|   7302454.51|23-Oct-19|15-Mar-20|         Pending|       Pending|
|OD10072|    C1177|    Osborne Fields|              Hawaii|           Shock Absorber|   33433|   3704874.96|15-Oct-19|17-Jan-20|       Completed|     Delivered|
|OD10124|    C1023|   Ilmio Jane M

In [14]:
highest_profit_shock_absorber = company_orders_df.select("CompanyID","CompanyName","ComponentToBeManufactured").where(col("ComponentToBeManufactured") == 'Shock Absorber').join(company_info_df).orderBy("ProfitMargin", ascending=False)

In [15]:
highest_profit_shock_absorber.show()

+---------+------------------+-------------------------+---------+------------------+---------------+---------------+--------------+------------+---------------+
|CompanyID|       CompanyName|ComponentToBeManufactured|CompanyID|       CompanyName|CompanyLocation| CompanyAddress|CompanyContact|ProfitMargin|EstablishedYear|
+---------+------------------+-------------------------+---------+------------------+---------------+---------------+--------------+------------+---------------+
|    C1023|   Ilmio Jane Moto|           Shock Absorber|    C1143|Nando Kewillswitch|         Hawaii|1 Oxford Avenue|    8083513813|   4995904.9|           1985|
|    C1064|       Zoom Dynamo|           Shock Absorber|    C1143|Nando Kewillswitch|         Hawaii|1 Oxford Avenue|    8083513813|   4995904.9|           1985|
|    C1047|      Damson Bikes|           Shock Absorber|    C1143|Nando Kewillswitch|         Hawaii|1 Oxford Avenue|    8083513813|   4995904.9|           1985|
|    C1177|    Osborne Field

##### Find the total number of companies that have ordered Fender

In [16]:
company_orders_df.filter(col("ComponentToBeManufactured") == "Fender").count()

40

##### Find the total number of Exhaust Pipes that have been ordered from North Carolina. 

In [17]:
company_orders_df.where(col("ComponentToBeManufactured") == "Exhaust Pipes").groupBy("CompanyLocation").sum("Quantity").show()

+--------------------+-------------+
|     CompanyLocation|sum(Quantity)|
+--------------------+-------------+
|           Minnesota|       109463|
|               Texas|       219363|
|        Pennsylvania|        54625|
|            Illinois|        50474|
|            Oklahoma|        24480|
|District of Columbia|        96422|
|          New Mexico|        24354|
|       West Virginia|        20745|
|             Georgia|       109078|
|            Michigan|        37061|
|            Virginia|        55303|
|      North Carolina|        42112|
|          New Jersey|        49215|
|             Alabama|        91219|
|       Massachusetts|        63774|
|           Louisiana|       128048|
|             Florida|        82370|
|      South Carolina|        62167|
|          California|       256804|
|            New York|       103112|
+--------------------+-------------+



##### Find the total count of all the orders that has been completed but not delivered yet.

In [18]:
company_orders_df.where((col("CompletionStatus") == "Completed") & (col("DeliveryStatus") == "Pending")).count()

246

##### Display all the orders whose status is still pending. Reduce the total cost of all such orders by 10%. Store them in a file called DiscountOrders.csv

In [19]:
new_comp_ord_df = company_orders_df.where(col("DeliveryStatus") == "Pending").withColumn("ReducedCost", expr("EstimatedCost - EstimatedCost * 0.10"))

In [20]:
final_df = new_comp_ord_df.select("CompanyID","CompanyName","EstimatedCost","ReducedCost")

In [21]:
#final_df.write.csv("final_csv", header=True)

In [22]:
final_df.show()

+---------+------------------+-------------+------------------+
|CompanyID|       CompanyName|EstimatedCost|       ReducedCost|
+---------+------------------+-------------+------------------+
|    C1108|       Raido Bikes|   9487867.48|       8539080.732|
|    C1035|   Ilmio Jane Moto|   4899369.32|       4409432.388|
|    C1121|          Lee Auto|   7932390.15|       7139151.135|
|    C1093|  Quakesons Motors|   6101829.61|       5491646.649|
|    C1166|         Rigmarole|   8704566.05|       7834109.445|
|    C1030|          Lee Auto|   9635904.98|       8672314.482|
|    C1135|   Ilmio Jane Moto|   6667701.97|       6000931.773|
|    C1121|          Lee Auto|    8501074.4|        7650966.96|
|    C1076|       Raido Bikes|   1676391.19|       1508752.071|
|    C1020|         Rigmarole|   1323523.14|       1191170.826|
|    C1064|       Zoom Dynamo|    5630252.2|        5067226.98|
|    C1073|   Ilmio Jane Moto|   3601243.24|       3241118.916|
|    C1165|  Quakesons Motors|   6893359

##### Based on the previous question, fetch the total cost to be paid by all the franchises having company name as Laser Wheels.

In [23]:
final_df.where(col("CompanyName") == "Laser Wheels").agg({"EstimatedCost":"sum"}).show()

+--------------------+
|  sum(EstimatedCost)|
+--------------------+
|1.1715670655999999E8|
+--------------------+



In [24]:
final_df.groupBy("CompanyName").sum().show()

+------------------+--------------------+--------------------+
|       CompanyName|  sum(EstimatedCost)|    sum(ReducedCost)|
+------------------+--------------------+--------------------+
|         Rigmarole|2.7541656048999995E8|2.4787490444099998E8|
|       Sano Wheels| 7.180358703999999E7| 6.462322833600001E7|
|    Osborne Fields|      1.3561335601E8|1.2205202040900001E8|
|Maino World Motors| 3.623841588900001E8| 3.261457430009999E8|
|       Wicker Auto|      1.5564068527E8|1.4007661674299994E8|
|      Steel Wheels|2.1206138532999998E8|1.9085524679699996E8|
|        Kioto Moto|      1.3924352236E8|1.2531917012399998E8|
|        Reo Bailio| 7.523934550000001E7|       6.771541095E7|
|       Zoom Dynamo|2.5168476422999993E8|2.2651628780699995E8|
|      Craiko Roads| 3.372631440600002E8| 3.035368296540001E8|
|      Laser Wheels|1.1715670655999999E8|     1.05441035904E8|
|     MotoLab Bikes|      1.4902537697E8|1.3412283927300002E8|
|  Quakesons Motors|3.3548822375000006E8|3.019394013749